In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# model_id = "/data1/lileilai/gpt_neox_20b/"
model_id = "/data1/lileilai/opt-6.7b/"
tokenizer = AutoTokenizer.from_pretrained(model_id,)
# we use device_map auto to automatically place all shards on the GPU to save CPU memory
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
print(f"model is loaded on device {model.device.type}")

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.61s/it]

model is loaded on device cuda


In [5]:
# tokenizer.pad_token_id

In [13]:
generate_kwargs = dict(max_new_tokens=100, do_sample=False, use_cache=True)
# tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

In [6]:
model = model.eval()

In [2]:
class Test(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(100,10)
    def forward(self, hidden_state):
        return self.fc1(hidden_state)

In [10]:
res.shape

torch.Size([4, 10])

In [14]:
input_sentences = ['In the last couple of days, a',
 'The New Jersey Department of Transportation is aware',
 'The New York Giants have a new head',
 'The New York Times has published its annual',
 'In a move that will likely make it',
 "The New York Giants' offensive linemen",
 'The Canadian Press has unanimously condemned the new',
 'The first time I saw the movie,'
]
batch_size = 8
inputs = input_sentences[: 8]

In [15]:
def generate():
    input_tokens = tokenizer.batch_encode_plus(input_sentences, return_tensors="pt", padding=True)
    for t in input_tokens:
        if torch.is_tensor(input_tokens[t]):
            input_tokens[t] = input_tokens[t].to("cuda:0")

    outputs = model.generate(**input_tokens, **generate_kwargs)

    input_tokens_lengths = [x.shape[0] for x in input_tokens.input_ids]
    output_tokens_lengths = [x.shape[0] for x in outputs]

    total_new_tokens = [o - i for i, o in zip(input_tokens_lengths, output_tokens_lengths)]
    outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return zip(inputs, outputs, total_new_tokens)

In [16]:
import time
import gc
import os
local_rank = int(os.getenv("LOCAL_RANK", "0"))
world_size = torch.cuda.device_count()

rank = local_rank
def print_rank0(*msg):
    if rank != 0:
        return
    print(*msg)

In [17]:
print("*** Running generate")
torch.cuda.empty_cache()
gc.collect()

print_rank0("*** Running benchmark")
# warm up
for i in range(1):
    _ = generate()
torch.cuda.synchronize()

# benchmark
t0 = time.time()
cycles = 5
total_new_tokens_generated = 0
for i in range(cycles):
    generated = generate()
    total_new_tokens_generated += sum(new_tokens for _, _, new_tokens in generated)
torch.cuda.synchronize()
throughput = (time.time() - t0) / (total_new_tokens_generated)
print((time.time() - t0 ) / cycles )
print_rank0(
        f"""
*** Performance stats:
Throughput per token including tokenize: {throughput*1000:.2f} msecs
"""
    )

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


*** Running generate
*** Running benchmark


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


3.21852560043335

*** Performance stats:
Throughput per token including tokenize: 4.02 msecs



In [1]:
output_data = [tokenizer.decode(d) for d in outputs]

NameError: name 'outputs' is not defined

In [22]:
output_data

['In the last couple of days, a lot of people have been asking me about the new book,',
 'The New Jersey Department of Transportation is aware of the issue and is working with the contractor to resolve it',
 'The New York Giants have a new head coach, and he’s a former NFL quarterback.\n',
 'The New York Times has published its annual list of the most influential people in the world, and the',
 'In a move that will likely make it easier for the company to raise money, the company said it',
 "The New York Giants' offensive linemen are a big reason why the team has been so successful in",
 'The Canadian Press has unanimously condemned the new law, saying it is “unacceptable” and “un',
 'The first time I saw the movie, I was in the theater and I was like, "Oh']

In [23]:
sum(consume_time) / len(consume_time)

1.801179575920105

In [6]:
d = ()

In [7]:
d = d + ((1,2),)

In [9]:
d = d + ((3, 4),)

In [10]:
d

((1, 2), (3, 4))